In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
db = DFDB('../trial/randomforest.pkl', auto_commit=False)

In [9]:
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

In [10]:
df_trial[['datetime','nfeatures', 'kfold', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].loc[0:38]

,datetime,nfeatures,kfold,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-11 14:48:38.289806,1071,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.606209,0.001154,2.254551,0.465856,1.648342,None
1,2019-05-11 16:28:02.970710,200,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.562026,0.000901,2.257117,0.452502,1.695091,None
2,2019-05-11 16:35:31.980435,190,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.558030,0.000921,2.250631,0.455314,1.692600,None
3,2019-05-11 16:42:51.099131,180,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.556696,0.000909,2.255689,0.459793,1.698994,None
4,2019-05-11 16:51:36.494608,170,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.551708,0.000874,2.248555,0.448692,1.696847,None
5,2019-05-11 17:01:36.317578,160,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.550463,0.000851,2.260096,0.449134,1.709633,None
6,2019-05-11 17:09:37.580623,150,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.547421,0.000996,2.248746,0.454676,1.701324,None
7,2019-05-11 17:18:15.631490,140,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.544545,0.000867,2.245204,0.452453,1.700659,None
8,2019-05-11 17:26:01.767176,130,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.541476,0.000847,2.250093,0.433307,1.708618,None
9,2019-05-11 17:33:10.404806,120,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.538778,0.000820,2.255671,0.450223,1.716894,None


In [ ]:
param_idx = 38
column_idx = 20
db_ = db
df_trial_ = df_trial
mytrial = []
columns = copy.deepcopy(df_trial_.loc[column_idx]['param']['columns'])
param = {'columns': ['q25_roll_std_100',
  'abs_q01_4',
  'q05_roll_std_1000',
  'abs_q25_5',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'median__roll_std',
  'min_roll_std_100',
  'max_to_min',
  'iqr_6'],
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'RandomForestRegressor',
  'init': {'n_estimators': 617,
   'max_depth': 9,
   'max_features': 0.6118189949437737,
   'min_samples_leaf': 0.10163450193406814,
   'random_state': 6370},
  'fit': {}}}
# param['algorithm']['init'] = {'logging_level': 'Silent', **param['algorithm']['init']}
# param['scaler']['init'] = {}
# param['algorithm']['fit'] = {}

selected_columns = EP.revert_rfe(df_train, param, columns, df_test, mytrial, start_columns=[columns[0]], limit=20, remark='start from top1 column 2th')
print(len(selected_columns))
for trial_i in mytrial:
    db_.insert(trial_i)
df_trial_ = db_.select()
df_trial_['kfold'] = df_trial_['param'].apply(lambda x: x['kfold'])

In [12]:
db_.commit()

In [10]:
df_trial[df_trial['remark']=='start from top1 column 2th'][['datetime','nfeatures', 'kfold', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,kfold,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
721,2019-05-24 02:17:27.366587,1,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.100329,0.013945,2.153255,0.549974,0.052926,start from top1 column 2th
722,2019-05-24 02:18:13.911269,1,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.100329,0.013945,2.153255,0.549974,0.052926,start from top1 column 2th
723,2019-05-24 02:19:00.849078,2,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.089561,0.014075,2.139190,0.559102,0.049629,start from top1 column 2th
724,2019-05-24 02:19:46.484385,3,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.085508,0.014216,2.134237,0.563771,0.048729,start from top1 column 2th
725,2019-05-24 02:20:56.632514,4,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.083400,0.014043,2.134460,0.566987,0.051060,start from top1 column 2th
726,2019-05-24 02:22:07.357038,4,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.086018,0.014122,2.137199,0.564114,0.051181,start from top1 column 2th
727,2019-05-24 02:23:18.005532,4,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.080193,0.013923,2.136026,0.569078,0.055833,start from top1 column 2th
728,2019-05-24 02:24:23.673581,4,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.085732,0.013954,2.139905,0.561115,0.054173,start from top1 column 2th
729,2019-05-24 02:25:33.995003,4,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.083631,0.013996,2.136416,0.570072,0.052785,start from top1 column 2th
730,2019-05-24 02:26:46.204570,4,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",start from top1 column 2th,2.086344,0.014059,2.139120,0.557766,0.052775,start from top1 column 2th


In [12]:
selected_columns = ['abs_q25_5', 'abs_q01_4', 'q25_roll_std_100']

In [13]:
mytrial =[]
columns_ = selected_columns

def objective(trial):
        
    n_estimators = trial.suggest_int('n_estimators', 300, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 16)
    max_features = trial.suggest_uniform('max_features', .6, 1)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0.1, 0.5)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':columns_,
        'kfold':{
            'n_splits': 8,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'RandomForestRegressor',
            'init':{
                "n_estimators":n_estimators,
                "max_depth":max_depth,
                "max_features":max_features,
                "min_samples_leaf":min_samples_leaf,
                "random_state":random_state,
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 724')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-24 15:27:20,506] Finished trial#0 resulted in value: 0.04596797733986259. Current best value is 0.04596797733986259 with parameters: {'n_estimators': 447, 'max_depth': 16, 'max_features': 0.7365110937441122, 'min_samples_leaf': 0.36247127988242445, 'random_state': 3122}.
[I 2019-05-24 15:27:36,120] Finished trial#1 resulted in value: 0.06887941392680968. Current best value is 0.04596797733986259 with parameters: {'n_estimators': 447, 'max_depth': 16, 'max_features': 0.7365110937441122, 'min_samples_leaf': 0.36247127988242445, 'random_state': 3122}.
[I 2019-05-24 15:28:05,861] Finished trial#2 resulted in value: 0.06899324480719538. Current best value is 0.04596797733986259 with parameters: {'n_estimators': 447, 'max_depth': 16, 'max_features': 0.7365110937441122, 'min_samples_leaf': 0.36247127988242445, 'random_state': 3122}.
[I 2019-05-24 15:28:10,919] Finished trial#3 resulted in value: 0.046241772843379436. Current best value is 0.04596797733986259 with parameters: {'n_es

[I 2019-05-24 15:33:18,006] Finished trial#28 resulted in value: 0.06924659022894485. Current best value is 0.04596797733986259 with parameters: {'n_estimators': 447, 'max_depth': 16, 'max_features': 0.7365110937441122, 'min_samples_leaf': 0.36247127988242445, 'random_state': 3122}.
[I 2019-05-24 15:33:34,544] Finished trial#29 resulted in value: 0.06861605769922961. Current best value is 0.04596797733986259 with parameters: {'n_estimators': 447, 'max_depth': 16, 'max_features': 0.7365110937441122, 'min_samples_leaf': 0.36247127988242445, 'random_state': 3122}.
[I 2019-05-24 15:33:49,988] Finished trial#30 resulted in value: 0.05016615598019726. Current best value is 0.04596797733986259 with parameters: {'n_estimators': 447, 'max_depth': 16, 'max_features': 0.7365110937441122, 'min_samples_leaf': 0.36247127988242445, 'random_state': 3122}.
[I 2019-05-24 15:33:57,331] Finished trial#31 resulted in value: 0.04630269976966492. Current best value is 0.04596797733986259 with parameters: {'n

[I 2019-05-24 15:39:02,499] Finished trial#56 resulted in value: 0.09920380320369064. Current best value is 0.04596325990225038 with parameters: {'n_estimators': 431, 'max_depth': 7, 'max_features': 0.9405250019225604, 'min_samples_leaf': 0.37067489496686057, 'random_state': 2377}.
[I 2019-05-24 15:39:16,891] Finished trial#57 resulted in value: 0.06898438451556076. Current best value is 0.04596325990225038 with parameters: {'n_estimators': 431, 'max_depth': 7, 'max_features': 0.9405250019225604, 'min_samples_leaf': 0.37067489496686057, 'random_state': 2377}.
[I 2019-05-24 15:39:27,055] Finished trial#58 resulted in value: 0.06720811994484825. Current best value is 0.04596325990225038 with parameters: {'n_estimators': 431, 'max_depth': 7, 'max_features': 0.9405250019225604, 'min_samples_leaf': 0.37067489496686057, 'random_state': 2377}.
[I 2019-05-24 15:39:35,724] Finished trial#59 resulted in value: 0.04609373183193041. Current best value is 0.04596325990225038 with parameters: {'n_es

[I 2019-05-24 15:43:42,852] Finished trial#84 resulted in value: 0.04608324690650183. Current best value is 0.04591762901889305 with parameters: {'n_estimators': 636, 'max_depth': 8, 'max_features': 0.7985820312366186, 'min_samples_leaf': 0.3814537907871066, 'random_state': 4537}.
[I 2019-05-24 15:43:58,828] Finished trial#85 resulted in value: 0.06909092984749249. Current best value is 0.04591762901889305 with parameters: {'n_estimators': 636, 'max_depth': 8, 'max_features': 0.7985820312366186, 'min_samples_leaf': 0.3814537907871066, 'random_state': 4537}.
[I 2019-05-24 15:44:05,536] Finished trial#86 resulted in value: 0.04637943253885268. Current best value is 0.04591762901889305 with parameters: {'n_estimators': 636, 'max_depth': 8, 'max_features': 0.7985820312366186, 'min_samples_leaf': 0.3814537907871066, 'random_state': 4537}.
[I 2019-05-24 15:44:19,877] Finished trial#87 resulted in value: 0.06912677364678207. Current best value is 0.04591762901889305 with parameters: {'n_estim

[I 2019-05-24 15:47:56,979] Finished trial#112 resulted in value: 0.04618905802310485. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 15:48:48,093] Finished trial#113 resulted in value: 0.11541444204860042. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 15:48:53,461] Finished trial#114 resulted in value: 0.04602049803864244. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 15:49:01,919] Finished trial#115 resulted in value: 0.046220295781800404. Current best value is 0.04571375838586438 with parameters: {

[I 2019-05-24 15:54:08,652] Finished trial#140 resulted in value: 0.0685812562526255. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 15:54:25,951] Finished trial#141 resulted in value: 0.06905535507516102. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 15:54:31,399] Finished trial#142 resulted in value: 0.046050172861454644. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 15:54:55,760] Finished trial#143 resulted in value: 0.06851250844493366. Current best value is 0.04571375838586438 with parameters: {'

[I 2019-05-24 15:58:10,987] Finished trial#168 resulted in value: 0.04624228958472103. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 15:58:19,897] Finished trial#169 resulted in value: 0.0461688603174103. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 15:58:29,091] Finished trial#170 resulted in value: 0.04606039311380741. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 15:58:36,048] Finished trial#171 resulted in value: 0.04585798512974414. Current best value is 0.04571375838586438 with parameters: {'n

[I 2019-05-24 16:01:29,974] Finished trial#196 resulted in value: 0.04600537735046243. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 16:01:34,391] Finished trial#197 resulted in value: 0.04658232230378543. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 16:01:42,495] Finished trial#198 resulted in value: 0.046170997917358736. Current best value is 0.04571375838586438 with parameters: {'n_estimators': 533, 'max_depth': 8, 'max_features': 0.8048279657898558, 'min_samples_leaf': 0.46247524370283755, 'random_state': 2549}.
[I 2019-05-24 16:02:27,109] Finished trial#199 resulted in value: 0.10836889597438845. Current best value is 0.04571375838586438 with parameters: {

In [14]:
for trial_i in mytrial:
    db.insert(trial_i)

In [15]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 724')&(df_trial['mae_diff']<.1)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
932,2019-05-24 15:30:07.425180,tune 724,3,2.085913,0.014133,2.135108,0.563899,0.049195
978,2019-05-24 15:39:02.412338,tune 724,3,2.093833,0.013753,2.140186,0.565232,0.046353
936,2019-05-24 15:31:01.069694,tune 724,3,2.088007,0.014027,2.142382,0.564311,0.054374
1035,2019-05-24 15:48:47.995309,tune 724,3,2.091159,0.013641,2.144966,0.566777,0.053807
1121,2019-05-24 16:02:27.001269,tune 724,3,2.095477,0.013710,2.145975,0.561377,0.050499
958,2019-05-24 15:35:36.508895,tune 724,3,2.102661,0.013855,2.148524,0.561096,0.045863
1043,2019-05-24 15:50:25.351733,tune 724,3,2.148939,0.013184,2.188534,0.534366,0.039595
945,2019-05-24 15:32:27.683424,tune 724,3,2.235681,0.013356,2.263090,0.557039,0.027409
1061,2019-05-24 15:53:36.909234,tune 724,3,2.235334,0.013387,2.263168,0.557442,0.027834
964,2019-05-24 15:36:47.028952,tune 724,3,2.232268,0.013115,2.263508,0.550577,0.031240


In [16]:
db.commit()

In [17]:
mytrial =[]
columns_ = selected_columns

def objective(trial):
        
    n_estimators = trial.suggest_int('n_estimators', 300, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 16)
    max_features = trial.suggest_uniform('max_features', .6, 1)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0.1, 0.5)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':columns_,
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'RandomForestRegressor',
            'init':{
                "n_estimators":n_estimators,
                "max_depth":max_depth,
                "max_features":max_features,
                "min_samples_leaf":min_samples_leaf,
                "random_state":random_state,
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 724 by stratified')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-24 16:07:51,388] Finished trial#0 resulted in value: 0.00911602625003897. Current best value is 0.00911602625003897 with parameters: {'n_estimators': 342, 'max_depth': 7, 'max_features': 0.7694987954401783, 'min_samples_leaf': 0.15908864357091246, 'random_state': 1551}.
[I 2019-05-24 16:08:00,668] Finished trial#1 resulted in value: 0.009469519143871067. Current best value is 0.00911602625003897 with parameters: {'n_estimators': 342, 'max_depth': 7, 'max_features': 0.7694987954401783, 'min_samples_leaf': 0.15908864357091246, 'random_state': 1551}.
[I 2019-05-24 16:08:11,053] Finished trial#2 resulted in value: 0.009371832745217964. Current best value is 0.00911602625003897 with parameters: {'n_estimators': 342, 'max_depth': 7, 'max_features': 0.7694987954401783, 'min_samples_leaf': 0.15908864357091246, 'random_state': 1551}.
[I 2019-05-24 16:08:13,097] Finished trial#3 resulted in value: 2.819944368112522e-05. Current best value is 2.819944368112522e-05 with parameters: {'n_

[I 2019-05-24 16:10:02,400] Finished trial#28 resulted in value: 3.779799435136363e-05. Current best value is 1.723416192895125e-05 with parameters: {'n_estimators': 504, 'max_depth': 5, 'max_features': 0.8731816223426658, 'min_samples_leaf': 0.4856404282390953, 'random_state': 9704}.
[I 2019-05-24 16:10:04,451] Finished trial#29 resulted in value: 2.8426103068881404e-05. Current best value is 1.723416192895125e-05 with parameters: {'n_estimators': 504, 'max_depth': 5, 'max_features': 0.8731816223426658, 'min_samples_leaf': 0.4856404282390953, 'random_state': 9704}.
[I 2019-05-24 16:10:07,433] Finished trial#30 resulted in value: 2.552340894649508e-05. Current best value is 1.723416192895125e-05 with parameters: {'n_estimators': 504, 'max_depth': 5, 'max_features': 0.8731816223426658, 'min_samples_leaf': 0.4856404282390953, 'random_state': 9704}.
[I 2019-05-24 16:10:11,234] Finished trial#31 resulted in value: 3.494057049555612e-05. Current best value is 1.723416192895125e-05 with para

[I 2019-05-24 16:11:51,223] Finished trial#56 resulted in value: 2.8882421318565077e-05. Current best value is 1.723416192895125e-05 with parameters: {'n_estimators': 504, 'max_depth': 5, 'max_features': 0.8731816223426658, 'min_samples_leaf': 0.4856404282390953, 'random_state': 9704}.
[I 2019-05-24 16:11:54,949] Finished trial#57 resulted in value: 1.9906686521186195e-05. Current best value is 1.723416192895125e-05 with parameters: {'n_estimators': 504, 'max_depth': 5, 'max_features': 0.8731816223426658, 'min_samples_leaf': 0.4856404282390953, 'random_state': 9704}.
[I 2019-05-24 16:12:00,219] Finished trial#58 resulted in value: 2.5007548076470557e-05. Current best value is 1.723416192895125e-05 with parameters: {'n_estimators': 504, 'max_depth': 5, 'max_features': 0.8731816223426658, 'min_samples_leaf': 0.4856404282390953, 'random_state': 9704}.
[I 2019-05-24 16:12:02,336] Finished trial#59 resulted in value: 4.1779321712945605e-05. Current best value is 1.723416192895125e-05 with p

[I 2019-05-24 16:13:31,448] Finished trial#84 resulted in value: 3.943893704174928e-05. Current best value is 1.4506532388059127e-05 with parameters: {'n_estimators': 378, 'max_depth': 16, 'max_features': 0.9479092596834686, 'min_samples_leaf': 0.4351850179202318, 'random_state': 5399}.
[I 2019-05-24 16:13:40,571] Finished trial#85 resulted in value: 0.010706912828789081. Current best value is 1.4506532388059127e-05 with parameters: {'n_estimators': 378, 'max_depth': 16, 'max_features': 0.9479092596834686, 'min_samples_leaf': 0.4351850179202318, 'random_state': 5399}.
[I 2019-05-24 16:13:42,516] Finished trial#86 resulted in value: 3.06576219364826e-05. Current best value is 1.4506532388059127e-05 with parameters: {'n_estimators': 378, 'max_depth': 16, 'max_features': 0.9479092596834686, 'min_samples_leaf': 0.4351850179202318, 'random_state': 5399}.
[I 2019-05-24 16:13:44,476] Finished trial#87 resulted in value: 3.1529423091246224e-05. Current best value is 1.4506532388059127e-05 with

[I 2019-05-24 16:15:15,241] Finished trial#112 resulted in value: 2.3777948250634294e-05. Current best value is 1.4506532388059127e-05 with parameters: {'n_estimators': 378, 'max_depth': 16, 'max_features': 0.9479092596834686, 'min_samples_leaf': 0.4351850179202318, 'random_state': 5399}.
[I 2019-05-24 16:15:17,700] Finished trial#113 resulted in value: 3.069943185318874e-05. Current best value is 1.4506532388059127e-05 with parameters: {'n_estimators': 378, 'max_depth': 16, 'max_features': 0.9479092596834686, 'min_samples_leaf': 0.4351850179202318, 'random_state': 5399}.
[I 2019-05-24 16:15:24,614] Finished trial#114 resulted in value: 0.009075756561655053. Current best value is 1.4506532388059127e-05 with parameters: {'n_estimators': 378, 'max_depth': 16, 'max_features': 0.9479092596834686, 'min_samples_leaf': 0.4351850179202318, 'random_state': 5399}.
[I 2019-05-24 16:15:28,849] Finished trial#115 resulted in value: 2.8434495702584255e-05. Current best value is 1.4506532388059127e-0

[I 2019-05-24 16:16:51,396] Finished trial#140 resulted in value: 0.009077008021876158. Current best value is 8.783395471131722e-06 with parameters: {'n_estimators': 424, 'max_depth': 14, 'max_features': 0.6577083446974634, 'min_samples_leaf': 0.4284834190006019, 'random_state': 6940}.
[I 2019-05-24 16:16:53,620] Finished trial#141 resulted in value: 2.1746636043202876e-05. Current best value is 8.783395471131722e-06 with parameters: {'n_estimators': 424, 'max_depth': 14, 'max_features': 0.6577083446974634, 'min_samples_leaf': 0.4284834190006019, 'random_state': 6940}.
[I 2019-05-24 16:16:55,467] Finished trial#142 resulted in value: 2.5150100454867147e-05. Current best value is 8.783395471131722e-06 with parameters: {'n_estimators': 424, 'max_depth': 14, 'max_features': 0.6577083446974634, 'min_samples_leaf': 0.4284834190006019, 'random_state': 6940}.
[I 2019-05-24 16:16:58,338] Finished trial#143 resulted in value: 2.3134606443028444e-05. Current best value is 8.783395471131722e-06 w

[I 2019-05-24 16:17:54,604] Finished trial#168 resulted in value: 3.769271734694475e-05. Current best value is 8.783395471131722e-06 with parameters: {'n_estimators': 424, 'max_depth': 14, 'max_features': 0.6577083446974634, 'min_samples_leaf': 0.4284834190006019, 'random_state': 6940}.
[I 2019-05-24 16:17:56,353] Finished trial#169 resulted in value: 2.9540950419342008e-05. Current best value is 8.783395471131722e-06 with parameters: {'n_estimators': 424, 'max_depth': 14, 'max_features': 0.6577083446974634, 'min_samples_leaf': 0.4284834190006019, 'random_state': 6940}.
[I 2019-05-24 16:17:58,301] Finished trial#170 resulted in value: 2.872767996388406e-05. Current best value is 8.783395471131722e-06 with parameters: {'n_estimators': 424, 'max_depth': 14, 'max_features': 0.6577083446974634, 'min_samples_leaf': 0.4284834190006019, 'random_state': 6940}.
[I 2019-05-24 16:18:01,919] Finished trial#171 resulted in value: 0.008525383807961905. Current best value is 8.783395471131722e-06 wit

[I 2019-05-24 16:19:09,553] Finished trial#196 resulted in value: 2.2214431878449782e-05. Current best value is 8.783395471131722e-06 with parameters: {'n_estimators': 424, 'max_depth': 14, 'max_features': 0.6577083446974634, 'min_samples_leaf': 0.4284834190006019, 'random_state': 6940}.
[I 2019-05-24 16:19:16,585] Finished trial#197 resulted in value: 0.009500223649380546. Current best value is 8.783395471131722e-06 with parameters: {'n_estimators': 424, 'max_depth': 14, 'max_features': 0.6577083446974634, 'min_samples_leaf': 0.4284834190006019, 'random_state': 6940}.
[I 2019-05-24 16:19:18,375] Finished trial#198 resulted in value: 2.87207127687043e-05. Current best value is 8.783395471131722e-06 with parameters: {'n_estimators': 424, 'max_depth': 14, 'max_features': 0.6577083446974634, 'min_samples_leaf': 0.4284834190006019, 'random_state': 6940}.
[I 2019-05-24 16:19:21,874] Finished trial#199 resulted in value: 2.7161516754785745e-05. Current best value is 8.783395471131722e-06 wit

In [18]:
for trial_i in mytrial:
    db.insert(trial_i)

In [19]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 724 by stratified')&(df_trial['mae_diff']<.1)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1167,2019-05-24 16:11:08.433473,tune 724 by stratified,3,2.089994,2.707575e-07,2.095076,0.000012,0.005082
1128,2019-05-24 16:08:30.616003,tune 724 by stratified,3,2.094151,1.737953e-07,2.099263,0.000023,0.005112
1233,2019-05-24 16:15:12.751099,tune 724 by stratified,3,2.094569,3.190716e-08,2.099646,0.000019,0.005078
1207,2019-05-24 16:13:40.475473,tune 724 by stratified,3,2.098339,1.231429e-06,2.103429,0.000022,0.005090
1131,2019-05-24 16:09:04.624325,tune 724 by stratified,3,2.099005,7.224498e-07,2.103852,0.000021,0.004847
1293,2019-05-24 16:18:01.812829,tune 724 by stratified,3,2.137259,1.087895e-05,2.141241,0.000080,0.003982
1177,2019-05-24 16:11:47.307938,tune 724 by stratified,3,2.150601,7.398815e-06,2.155149,0.000101,0.004548
1122,2019-05-24 16:07:51.299929,tune 724 by stratified,3,2.150976,2.215598e-06,2.155206,0.000043,0.004230
1162,2019-05-24 16:10:45.993948,tune 724 by stratified,3,2.160075,2.413575e-05,2.164362,0.000072,0.004287
1288,2019-05-24 16:17:51.093464,tune 724 by stratified,3,2.235894,1.051608e-06,2.239060,0.000016,0.003166


In [20]:
db.commit()